# Fraud Detection Project

In [49]:
import pandas as pd
import numpy as np
from datetime import datetime
from neo4j import GraphDatabase
import time

## 50MB Dataset

In [33]:
# Customer 50MB
load_customer = """
    LOAD CSV WITH HEADERS
        FROM 'https://media.githubusercontent.com/media/miadalavinezhad/Fraud_detection_neo4j/main/new_datasets/data_sets_50/customer.csv' AS row
    CALL {
    WITH row
    CREATE (c:Customer {customer_id: row.CUSTOMER_ID, 
                        x_customer_id: row.x_customer_id, 
                        y_customer_id:  row.y_customer_id, 
                        mean_amount: row.mean_amount, 
                        std_amount: row.std_amount, 
                        mean_nb_tx_per_day: row.mean_nb_tx_per_day, 
                        available_terminals: row.available_terminals, 
                        nb_terminals: row.nb_terminals})
    } IN TRANSACTIONS OF 200 ROWS
    
"""

In [34]:
# Terminal
load_terminal = """
        LOAD CSV WITH HEADERS
            FROM 'https://media.githubusercontent.com/media/miadalavinezhad/Fraud_detection_neo4j/main/new_datasets/data_sets_50/terminal.csv' AS row
        CALL {
        WITH row
        CREATE (t:Terminal {terminal_id: row.TERMINAL_ID, 
                x_terminal_id: row.x_terminal_id, y_terminal_id: row.y_terminal_id})
        } IN TRANSACTIONS OF 200 ROWS
"""

In [35]:
load_transaction = """
        LOAD CSV WITH HEADERS
            FROM 'https://media.githubusercontent.com/media/miadalavinezhad/Fraud_detection_neo4j/main/new_datasets/data_sets_50/transaction.csv' AS row
        CALL {
        WITH row
        MATCH (c:Customer {customer_id: row.CUSTOMER_ID})
        MATCH (t:Terminal {terminal_id: row.TERMINAL_ID})
        CREATE (c)-[:MADE]->(tx:Transaction {transaction_id: row.TRANSACTION_ID, 
                                            transaction_datetime: row.TX_DATETIME, 
                                            transaction_amount: row.TX_AMOUNT})
        CREATE (t)-[:PROCESS]->(tx)
        CREATE (c)-[:USE]->(t)
        } IN TRANSACTIONS OF 500 ROWS
"""

## 100MB Dataset

In [121]:
# Customer 100MB
load_customer = """
    LOAD CSV WITH HEADERS
        FROM 'https://media.githubusercontent.com/media/miadalavinezhad/Fraud_detection_neo4j/main/new_datasets/data_sets_100/customer.csv' AS row
    CALL {
    WITH row
    CREATE (c:Customer {customer_id: row.CUSTOMER_ID, 
                        x_customer_id: row.x_customer_id, 
                        y_customer_id:  row.y_customer_id, 
                        mean_amount: row.mean_amount, 
                        std_amount: row.std_amount, 
                        mean_nb_tx_per_day: row.mean_nb_tx_per_day, 
                        available_terminals: row.available_terminals, 
                        nb_terminals: row.nb_terminals})
    } IN TRANSACTIONS OF 200 ROWS
    
"""

In [122]:
# Terminal 100MB
load_terminal = """
        LOAD CSV WITH HEADERS
            FROM 'https://media.githubusercontent.com/media/miadalavinezhad/Fraud_detection_neo4j/main/new_datasets/data_sets_100/terminal.csv' AS row
        CALL {
        WITH row
        CREATE (t:Terminal {terminal_id: row.TERMINAL_ID, 
                x_terminal_id: row.x_terminal_id, y_terminal_id: row.y_terminal_id})
        } IN TRANSACTIONS OF 200 ROWS
"""

In [123]:
load_transaction = """
        LOAD CSV WITH HEADERS
            FROM 'https://media.githubusercontent.com/media/miadalavinezhad/Fraud_detection_neo4j/main/new_datasets/data_sets_100/transaction.csv' AS row
        CALL {
        WITH row
        MATCH (c:Customer {customer_id: row.CUSTOMER_ID})
        MATCH (t:Terminal {terminal_id: row.TERMINAL_ID})
        CREATE (c)-[:MADE]->(tx:Transaction {transaction_id: row.TRANSACTION_ID, 
                                            transaction_datetime: row.TX_DATETIME, 
                                            transaction_amount: row.TX_AMOUNT})
        CREATE (t)-[:PROCESS]->(tx)
        CREATE (c)-[:USE]->(t)
        } IN TRANSACTIONS OF 500 ROWS
"""

## 200MB Dataset

In [133]:
# Customer 200MB
load_customer = """
    LOAD CSV WITH HEADERS
        FROM 'https://media.githubusercontent.com/media/miadalavinezhad/Fraud_detection_neo4j/main/new_datasets/data_sets_200/customer.csv' AS row
    CALL {
    WITH row
    CREATE (c:Customer {customer_id: row.CUSTOMER_ID, 
                        x_customer_id: row.x_customer_id, 
                        y_customer_id:  row.y_customer_id, 
                        mean_amount: row.mean_amount, 
                        std_amount: row.std_amount, 
                        mean_nb_tx_per_day: row.mean_nb_tx_per_day, 
                        available_terminals: row.available_terminals, 
                        nb_terminals: row.nb_terminals})
    } IN TRANSACTIONS OF 200 ROWS
    
"""

In [134]:
# Terminal 200MB
load_terminal = """
        LOAD CSV WITH HEADERS
            FROM 'https://media.githubusercontent.com/media/miadalavinezhad/Fraud_detection_neo4j/main/new_datasets/data_sets_200/terminal.csv' AS row
        CALL {
        WITH row
        CREATE (t:Terminal {terminal_id: row.TERMINAL_ID, 
                x_terminal_id: row.x_terminal_id, y_terminal_id: row.y_terminal_id})
        } IN TRANSACTIONS OF 200 ROWS
"""

In [135]:
load_transaction = """
        LOAD CSV WITH HEADERS
            FROM 'https://media.githubusercontent.com/media/miadalavinezhad/Fraud_detection_neo4j/main/new_datasets/data_sets_200/transaction.csv' AS row
        CALL {
        WITH row
        MATCH (c:Customer {customer_id: row.CUSTOMER_ID})
        MATCH (t:Terminal {terminal_id: row.TERMINAL_ID})
        CREATE (c)-[:MADE]->(tx:Transaction {transaction_id: row.TRANSACTION_ID, 
                                            transaction_datetime: row.TX_DATETIME, 
                                            transaction_amount: row.TX_AMOUNT})
        CREATE (t)-[:PROCESS]->(tx)
        CREATE (c)-[:USE]->(t)
        } IN TRANSACTIONS OF 500 ROWS
"""

## Constrains

In [136]:
# Adding unique constraint to ID
customer_constraint = """
                        CREATE CONSTRAINT customer IF NOT EXISTS
                        FOR (c:Customer) REQUIRE c.customer_id IS UNIQUE
"""
terminal_constraint = """
                        CREATE CONSTRAINT terminal IF NOT EXISTS
                        FOR (t:Terminal) REQUIRE t.terminal_id IS UNIQUE
"""
transaction_constraint = """
                        CREATE CONSTRAINT transaction IF NOT EXISTS
                        FOR (tx:Transaction) REQUIRE tx.transaction_id IS UNIQUE
"""

## Set DB driver

In [137]:
URI = "bolt://localhost:7687/"
AUTH = ("neo4j", "12345678")

# Session
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

    with driver.session() as session:
        session.run(load_customer).data()
        session.run(customer_constraint)

        session.run(load_terminal).data()
        session.run(terminal_constraint)
        
        session.run(load_transaction).data()
        session.run(transaction_constraint)
        

## Scripts of operations

In [117]:
query_a = """
            MATCH (c:Customer)-[:MADE]->(tr:Transaction)
            WITH c, datetime(tr.transaction_datetime) AS transactionDate, tr.transaction_amount AS amount
            ORDER BY transactionDate DESC
            WITH c, COLLECT(transactionDate)[0] AS lastTransactionDate 
            WITH c, lastTransactionDate.month AS lastMonth
            
            MATCH (c)-[:MADE]->(tr:Transaction)
            WHERE datetime(tr.transaction_datetime).month = lastMonth
            WITH c, lastMonth, SUM(toInteger(tr.transaction_amount)) AS lastMonthSpending, COUNT(tr) as lastMonthFrequency
            
            MATCH (c)-[:MADE]->(tr:Transaction)
            WHERE datetime(tr.transaction_datetime).month < lastMonth
            WITH c, lastMonthSpending, lastMonthFrequency, (SUM(toInteger(tr.transaction_amount)) / 5) AS usualSpending, (COUNT(tr) / 5) AS usualFrequency
            
            WHERE lastMonthSpending < usualSpending AND lastMonthFrequency < usualFrequency
            RETURN c.customer_id AS customer_id, lastMonthSpending, usualSpending, lastMonthFrequency, usualFrequency;
"""

query_b = """
            MATCH (t:Terminal)-[:PROCESS]->(tr:Transaction)
            WITH t, datetime(tr.transaction_datetime) AS transactionDate, tr.transaction_amount AS amount
            ORDER BY transactionDate DESC 
            WITH t, COLLECT(transactionDate)[0] AS lastTransactionDate
            WITH t, lastTransactionDate.month AS lastMonth

            MATCH (t)-[:PROCESS]->(tr:Transaction)
            WHERE datetime(tr.transaction_datetime).month = lastMonth
            WITH t, MAX(tr.transaction_amount) AS maxImport, lastMonth

            MATCH (t)-[:PROCESS]->(tr:Transaction)
            WHERE datetime(tr.transaction_datetime).month = lastMonth
            AND toInteger(tr.transaction_amount) > toInteger(maxImport) * 1.2
            RETURN t.terminal_id AS terminalId, tr.transaction_id AS transactionId, tr.transaction_amount AS transactionAmount , maxImport, 1.2 * toInteger(maxImport) AS threshold;

"""

query_c = """
            MATCH (u1:Customer)-[:USE*4]-(u2:Customer) 
            WHERE u1.customer_id <> u2.customer_id 
            RETURN distinct u1.customer_id, u2.customer_id
    """

query_e = """
            MATCH (tr:Transaction) 
            WITH tr.transaction_period AS period, COUNT(tr) AS totalTransactions, 
            AVG(CASE WHEN tr.transaction_security < 3 THEN 1.0 ELSE 0.0 END) AS avgFraudulent
            RETURN period, totalTransactions, avgFraudulent
"""


In [138]:
# Session
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

    with driver.session() as session:
        start = time.time()
        result_a = session.run(query_a)
        print(len([row.data() for row in result_a]))
        print('query a time:', time.time() - start)
        
        start = time.time()
        result_b = session.run(query_b)
        print(len([row.data() for row in result_b]))
        print('query b time:',time.time() - start)

        # start = time.time()
        # result_c = session.run(query_c)
        # print(len([row.data() for row in result_c]))
        # print('query c time:',time.time() - start)

TransientError: {code: Neo.TransientError.General.MemoryPoolOutOfMemoryError} {message: The allocation of an extra 2.0 MiB would use more than the limit 716.8 MiB. Currently using 716.0 MiB. dbms.memory.transaction.total.max threshold reached}

In [ ]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

    with driver.session() as session:
        start = time.time()
        result_c = session.run(query_c)
        print(len([row.data() for row in result_c]))
        print('query c time:',time.time() - start)

In [ ]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

    with driver.session() as session:
        start = time.time()
        result_e = session.run(query_e)
        print(len([row.data() for row in result_e]))
        print('query e time:',time.time() - start)

### Extending the Database

In [93]:
def add_time_period(session):
    query = """
            MATCH (tr:Transaction)
            CALL {
            WITH tr
            WITH
            CASE
                WHEN datetime(tr.transaction_datetime).hour > 5 AND datetime(tr.transaction_datetime).hour <= 12 THEN 'Morning'
                WHEN datetime(tr.transaction_datetime).hour > 12 AND datetime(tr.transaction_datetime).hour <= 17 THEN 'Afternoon'
                WHEN datetime(tr.transaction_datetime).hour > 17 AND datetime(tr.transaction_datetime).hour <= 21 THEN 'Evening'
                ELSE 'Night'
            END AS day_period, tr
    
            SET tr.transaction_period = day_period
            } IN TRANSACTIONS OF 500 ROWS
    """
    session.run(query)

def add_product_category(session):
    query = """
            MATCH (tr:Transaction)
            CALL {
            WITH tr
            WITH
            CASE
                WHEN tr.transaction_amount >= 0 AND tr.transaction_amount < 25 THEN 'Food'
                WHEN tr.transaction_amount >= 25 AND tr.transaction_amount < 50 THEN 'Clothing'
                WHEN tr.transaction_amount >= 50 AND tr.transaction_amount < 55 THEN 'Consumable'
                ELSE 'High-Tech'
            END AS category, tr
    
            SET tr.transaction_category = category
            } IN TRANSACTIONS OF 500 ROWS 
    """
    
    session.run(query)

def add_security(session):
    query = """
            MATCH (tr:Transaction)
            CALL {
            WITH tr
            SET tr.transaction_security = FLOOR(rand() * 5) + 1
            } IN TRANSACTIONS OF 500 ROWS
    """
    
    session.run(query)

def buying_friends(session):
    query = """
            MATCH (c1:Customer)-[:MADE]-(tr1:Transaction)-[:PROCESS]-(t1:Terminal),
                  (c2:Customer)-[:MADE]-(tr2:Transaction)-[:PROCESS]-(t2:Terminal)
            WHERE c1 <> c2
            WITH c1, c2, t1, t2, tr1, tr2
            WHERE t1.terminal_id = t2.terminal_id
            WITH c1, c2, t1, COUNT(tr1) AS c1_transactions, COUNT(tr2) AS c2_transactions, AVG(tr1.transaction_security) AS c1_avg_security, AVG(tr2.transaction_security) AS c2_avg_security
            WHERE c1_transactions > 3 AND c2_transactions > 3 AND ABS(c1_avg_security - c2_avg_security) < 1
            
            MERGE (c1)-[:BUYING_FRIENDS]->(c2)
    """
    
    session.run(query)


In [128]:
# Session
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session() as session:
        start = time.time()
        add_time_period(session)
        print(time.time() - start)

206.99079990386963


In [129]:
# Session
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session() as session:
        start = time.time()
        add_product_category(session)
        print(time.time() - start)

183.94830012321472


In [130]:
# Session
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session() as session:
        start = time.time()
        add_security(session)
        print(time.time() - start)

153.20670318603516


In [132]:
# Session
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session() as session:
        start = time.time()
        buying_friends(session) 
        print(time.time() - start)

KeyboardInterrupt: 